In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from pprint import pprint
from typing import Dict, Any
from google import genai
from dotenv import load_dotenv
from cfg_builder import *
from prompts import *

In [3]:
load_dotenv()
client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

### Retrieve codebase

In [4]:
codebase = json.load(open("../data/sample_codebase.json"))
list(codebase.keys())

['/project/app/main.py',
 '/project/app/utils/data_loader.py',
 '/project/app/utils/math_ops.py',
 '/project/app/services/processor.py',
 '/project/app/services/report.py',
 '/project/app/tests/test_math_ops.py',
 '/project/app/tests/test_processor.py']

### CFG Builder

In [5]:
generated_cfg = build_cfg(codebase)
generated_cfg

{'project': 'generated_project',
 'files': {'/project/app/main.py': {'functions': {'main': {'entry_node': 'main.py.main.entry_1',
     'exit_node': 'main.py.main.exit_2',
     'nodes': [{'id': 'main.py.main.entry_1',
       'type': 'ENTRY',
       'text': '',
       'line': None,
       'indent': 0,
       'metadata': {}},
      {'id': 'main.py.main.assign_3',
       'type': 'ASSIGN',
       'text': "numbers = load_numbers('data/input.txt')",
       'line': 6,
       'indent': 0,
       'metadata': {}},
      {'id': 'main.py.main.call_4',
       'type': 'CALL',
       'text': "load_numbers('data/input.txt')",
       'line': 6,
       'indent': 0,
       'metadata': {'called_function': 'load_numbers'}},
      {'id': 'main.py.main.expr_5',
       'type': 'EXPR',
       'text': "print('Loaded numbers:', numbers)",
       'line': 7,
       'indent': 0,
       'metadata': {}},
      {'id': 'main.py.main.call_6',
       'type': 'CALL',
       'text': "print('Loaded numbers:', numbers)",
    

In [6]:
out_path = "../data/generated_cfg.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(generated_cfg, f, indent=2)
print("Saved CFG to", out_path)

# Print a small preview (first file)
for i in range(len(generated_cfg["files"])):
    first = list(generated_cfg["files"].keys())[i]
    print("Preview file:", first)
    print("Functions:", list(generated_cfg["files"][first]["functions"].keys()))

Saved CFG to ../data/generated_cfg.json
Preview file: /project/app/main.py
Functions: ['main', '<module>']
Preview file: /project/app/utils/data_loader.py
Functions: ['load_numbers', '<module>']
Preview file: /project/app/utils/math_ops.py
Functions: ['compute_average', 'compute_factorial', '<module>']
Preview file: /project/app/services/processor.py
Functions: ['NumberProcessor.__init__', 'NumberProcessor.process', 'NumberProcessor.unstable_sort', '<module>']
Preview file: /project/app/services/report.py
Functions: ['generate_report', '<module>']
Preview file: /project/app/tests/test_math_ops.py
Functions: ['test_compute_average_basic', 'test_compute_average_empty', 'test_factorial_zero', '<module>']
Preview file: /project/app/tests/test_processor.py
Functions: ['test_processor_flow', 'test_sort', '<module>']


### LLM Call

In [7]:
debug_prompt = get_debug_prompt()
input_sample = get_sample_input()
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[debug_prompt + '\nINPUT:\n' + input_sample],
    config={
        # "response_mime_type": "application/json",
        # 'response_schema': 
    }
)

In [8]:
pprint(response.text)

('1. Root Cause Summary\n'
 '\n'
 'The true root cause of the failure is an incorrect modification to the loop '
 "range in the `analyze_numbers` function. The developer's change from `for i "
 'in range(len(nums))` to `for i in range(len(nums) - 1)` causes the loop (CFG '
 'node `a2`) to iterate one fewer time than intended. This premature '
 'truncation of the loop prevents the code from evaluating all necessary '
 'prefixes of `nums`, specifically missing the `i` value where the `avg > '
 'threshold` condition might have been met. As a result, the loop completes '
 'without finding a suitable index, leading the function to incorrectly '
 'proceed to node `a6` and return `-1`, which directly causes the '
 '`AssertionError`. The `IndexError` and `i = -1` observed in the trace are '
 "likely anomalous side-effects or a misrepresentation of the Python runtime's "
 'internal state when encountering this logically incorrect loop boundary, '
 'rather than the primary functional bug.\n'
 '\

In [9]:
# if response.text is not None:
#     json_output = json.loads(response.text)

### Parse txt to CFG

In [11]:
with open("../data/cfg_format2.json", "r") as f:
    cfg_output = json.load(f)
cfg_output


{'nodes': [{'id': 'unique_node_id',
   'parent_block': 'id_of_parent_block_node',
   'metadata': {'path': '/full/path/to/file.py',
    'type': 'statement_type',
    'text': 'raw source code of the statement',
    'condition': '...',
    'iterator': '...',
    'called_function': '...',
    'exception_type': '...',
    'returns_value': True,
    'is_unreachable': False,
    'variables_read': ['x', 'y'],
    'variables_written': ['z']}}],
 'edges': [{'id': 'edge_id_1', 'from': 'node_id_1', 'to': 'node_id_2'},
  {'id': 'edge_id_2', 'from': 'node_id_2', 'to': 'external_node'}]}

In [ ]:
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[debug_prompt + '\nINPUT:\n' + input_sample],
    config={
        # "response_mime_type": "application/json",
        # 'response_schema': 
    }
)